# Lesson 4: Building Agents

https://github.com/run-llama/llama_index/blob/767de070b231fb328b6c0640c2e002c9c7af0a83/docs/docs/examples/agent/custom_agent.ipynb#L12

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

## 1. Setup an agent with tools loaded from yaml file

In [3]:
import yaml
import importlib
from llama_index.core.tools import FunctionTool

class Tool:
    def __init__(self, config):
        self.name = config['name']
        self.module = config['module']
        self.function = config['function']
        self.instance = self.load_tool()

    def load_tool(self):
        module = importlib.import_module(self.module)
        tool_func = getattr(module, self.function)
        return tool_func

def load_tools_config(file_path):
    with open(file_path, 'r') as file:
        tools_config = yaml.safe_load(file)
    return tools_config

In [4]:
tools_config = load_tools_config('tools.yaml')

In [5]:
tools_config

{'tools': [{'name': 'add', 'module': 'tools.sample_tools', 'function': 'add'},
  {'name': 'subtract', 'module': 'tools.sample_tools', 'function': 'subtract'},
  {'name': 'multiply', 'module': 'tools.sample_tools', 'function': 'multiply'},
  {'name': 'divide', 'module': 'tools.sample_tools', 'function': 'divide'},
  {'name': 'search',
   'module': 'tools.search_tools',
   'function': 'search_ddg'}]}

In [6]:
all_tools = [Tool(config) for config in tools_config["tools"]]

In [7]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [8]:
initial_tools = [ FunctionTool.from_defaults(fn=fn.instance) for fn in all_tools]

In [9]:
len(initial_tools)

5

In [10]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools, 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [11]:
# response = agent.query(
#     "Think step by step, How much is 2 + 2 * 3"
# )

In [12]:
# response = agent.query("How much is 3.14159 * 2.334")
# print(str(response))

In [13]:
# response = agent.query("How much is 10 / 3 * 3")
# print(str(response)) 

In [14]:
# response = agent.query("Think step by step, how much is 10 / 3 * 3")
# print(str(response)) 

In [15]:
agents_config = yaml.safe_load(open('agents.yaml', 'r'))

In [16]:
agents_config

{'agents': [{'name': 'mathematician',
   'role': 'Answer mathematics questions',
   'prompt': 'You are an expert in mathematics, you will answer questions related to maths.',
   'tools': ['add', 'subtract', 'multiply', 'divide'],
   'verbose': True},
  {'name': 'online_research',
   'role': 'You know how to search in duckduckgo',
   'prompt': 'You are an expert in online search',
   'tools': ['search'],
   'verbose': True},
  {'name': 'oracle',
   'role': "Answer questions about people's life and death",
   'prompt': "You are the Oracle and you make up stories about people's life and death.",
   'verbose': True}]}

In [17]:
agents = dict()

In [18]:
from llama_index.core.bridge.pydantic import PrivateAttr
from llama_index.core.tools import BaseTool, QueryEngineTool



In [19]:
from llama_index.core.query_engine import RouterQueryEngine


In [20]:
from llama_index.core.agent import (
    CustomSimpleAgentWorker,
    Task,
    AgentChatResponse,
)
from typing import Dict, Any, List, Tuple, Optional
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core import ChatPromptTemplate, PromptTemplate
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.bridge.pydantic import Field, BaseModel

In [21]:

from llama_index.core.llms import ChatMessage, MessageRole

DEFAULT_PROMPT_STR = """
Given previous question/response pairs, please determine if an error has occurred in the response, and suggest \
a modified question that will not trigger the error.

Examples of modified questions:
- The question itself is modified to elicit a non-erroneous response
- The question is augmented with context that will help the downstream system better answer the question.
- The question is augmented with examples of negative responses, or other negative questions.

An error means that either an exception has triggered, or the response is completely irrelevant to the question.

Please return the evaluation of the response in the following JSON format.

"""


def get_chat_prompt_template(
    system_prompt: str, current_reasoning: Tuple[str, str]
) -> ChatPromptTemplate:
    system_msg = ChatMessage(role=MessageRole.SYSTEM, content=system_prompt)
    messages = [system_msg]
    for raw_msg in current_reasoning:
        if raw_msg[0] == "user":
            messages.append(
                ChatMessage(role=MessageRole.USER, content=raw_msg[1])
            )
        else:
            messages.append(
                ChatMessage(role=MessageRole.ASSISTANT, content=raw_msg[1])
            )
    return ChatPromptTemplate(message_templates=messages)

class ResponseEval(BaseModel):
    """Evaluation of whether the response has an error."""

    has_error: bool = Field(
        ..., description="Whether the response has an error."
    )
    new_question: str = Field(..., description="The suggested new question.")
    explanation: str = Field(
        ...,
        description=(
            "The explanation for the error as well as for the new question."
            "Can include the direct stack trace as well."
        ),
    )


In [22]:
class SimpleAgentWorker(CustomSimpleAgentWorker):
    """Agent worker that adds a retry layer on top of a router.

    Continues iterating until there's no errors / task is done.

    """

    prompt_str: str = Field(default=DEFAULT_PROMPT_STR)
    role_prompt: str = Field(default="You are a AI assistant.")
    max_iterations: int = Field(default=3)
    __fields_set__: set =  {"prompt_str", "max_iterations","role_prompt"}
    _router_query_engine: RouterQueryEngine = PrivateAttr()
    

    def __init__(self, **kwargs: Any) -> None:
        """Initialize agent worker."""
        self.tools = []
        self.role_prompt = kwargs.get("role_prompt", "") + DEFAULT_PROMPT_STR
        self._router_query_engine = RouterQueryEngine(
            selector=PydanticSingleSelector.from_defaults(),
            query_engine_tools=self.tools,
            verbose=kwargs.get("verbose", False),
        )
        super().__init__(
            tools=[],
            **kwargs,
        )

    def _initialize_state(self, task: Task, **kwargs: Any) -> Dict[str, Any]:
        """Initialize state."""
        return {"count": 0, "current_reasoning": []}

    def _run_step(
        self, state: Dict[str, Any], task: Task, input: Optional[str] = None
    ) -> Tuple[AgentChatResponse, bool]:
        """Run step.

        Returns:
            Tuple of (agent_response, is_done)

        """
        if "new_input" not in state:
            new_input = task.input
        else:
            new_input = state["new_input"]

        if self.verbose:
            print(f"> Querying engine: {new_input}")
        
        if self.verbose:
            print(f"> Prompt: {self.role_prompt}")
        response = self.llm.complete(self.role_prompt + new_input)

            
        # append to current reasoning
        state["current_reasoning"].extend(
            [("user", new_input), ("assistant", str(response))]
        )
        is_done = True
        

        if self.verbose:
            print(f"> Question: {new_input}")
            print(f"> Response: {response}")
            # print(f"> Response eval: {response_eval.dict()}")

        # return response
        return AgentChatResponse(response=str(response)), is_done

    def _finalize_task(self, state: Dict[str, Any], **kwargs) -> None:
        """Finalize task."""
        # nothing to finalize here
        # this is usually if you want to modify any sort of
        # internal state beyond what is set in `_initialize_state`
        pass


In [23]:
for agent_config in agents_config["agents"]:
    initial_tools = [ FunctionTool.from_defaults(fn=fn.instance) for fn in all_tools if fn.name in agent_config.get("tools",[])]
    if len(initial_tools) == 0:
        agent_worker = SimpleAgentWorker(llm=llm, role_prompt=agent_config["prompt"], can_delegate=agent_config.get("can_delegate",False), verbose=True)
    else:
        agent_worker = FunctionCallingAgentWorker.from_tools(
            initial_tools, 
            llm=llm, 
            verbose=True
        )
    agent = AgentRunner(agent_worker)
    agents[agent_config["name"]] = agent
    

In [24]:
agents.keys()

dict_keys(['mathematician', 'online_research', 'oracle'])

In [25]:
# agents["oracle"].query(" When will I get married?")


In [26]:
# agents["oracle"].state

In [27]:
# agents["mathematician"].query("How much is 10 / 3 * 3")

In [28]:
# agents["mathematician"].state

In [29]:
# agents['ceo_expert'].query("What is the best way to increase revenue?")

In [30]:
class Orchestrator:
    system_prompt = "You are the orchestrator. You can ask any of the agents: [{agents_list}] and forward the response to the user Given the follow question from the user:\n"
    def __init__(self, llm, agents, **kwargs):
        self.llm = llm
        self.agents = agents
        self.agents_config = kwargs.get("agents_config", {})
        self.verbose = kwargs.get("verbose", False)
        
    def query(self, query):
        prompt = self.system_prompt.format(agents_list=[agent["name"]+ ":" + agent["role"] for agent in self.agents_config["agents"]])
        step = self.llm.complete( prompt + query +  "\n\nRespond with the name of the agent to call and the query to forward to the agent in the following format: 'agent_name: query'")
        agent_name, agent_query = str(step).split(":")
        if self.verbose:
            print(f"Forwarding the query to the agent {agent_name} with the query: {agent_query}")
        response = self.query_agent(agent_name.strip(), agent_query.strip())
        if self.verbose:
            print(f"Agent {agent_name} responded with: {response}")
        eval_response = self.eval_response(query, agent_name.strip(), agent_query.strip(), response)
        if self.verbose:
            print(f"Response evaluation: {eval_response}")
        return response, eval_response
        
    
    def eval_response(self, task, agent_name, query, response):
        system_prompt = f"Given the follow question from the user:\n{task}\n\nThe response from the agent {agent_name} to the query {query} is:\n{response}\n\nPlease evaluate the response in the following format: 'has_error: new_question: explanation'"
        return self.llm.complete(system_prompt)

    def query_agent(self, agent_name, query):
        return self.agents[agent_name].query(query)

In [31]:
director = Orchestrator(llm, agents, agents_config=agents_config, verbose=True)

In [32]:
# director.query("Ask the oracle When will I get married?")

In [33]:
# director.query("How much is 10 / 3 * 3")

In [34]:
director.query("What was the dollar price on ethereum back in May 6th 2024?")

Forwarding the query to the agent online_research with the query:  Search for the dollar price of ethereum on May 6th, 2024.
Added user message to memory: Search for the dollar price of ethereum on May 6th, 2024.
=== Calling Function ===
Calling function: search_ddg with args: {"query": "Ethereum price on May 6th, 2024 in USD"}
=== Function Output ===
[{'title': 'Ethereum USD (ETH-USD) Stock Historical Prices & Data - Yahoo Finance', 'href': 'https://finance.yahoo.com/quote/ETH-USD/history/', 'body': "Discover historical prices for ETH-USD stock on Yahoo Finance. View daily, weekly or monthly format back to when Ethereum USD stock was issued. News. Today's news; US; ... May 6, 2024: 3,137.51: ..."}, {'title': 'Ethereum USD (ETH-USD) Price History & Historical Data - Yahoo Finance', 'href': 'https://ca.finance.yahoo.com/quote/ETH-USD/history/', 'body': '2,298.89. 7,277,068,110. *Close price adjusted for splits. **Adjusted close price adjusted for splits and dividend and/or capital gain 

(Response(response='assistant: The price of Ethereum on May 6th, 2024, was $3,137.51 USD.', source_nodes=[], metadata=None),
 CompletionResponse(text="has_error: No\nnew_question: N/A\nexplanation: The response provided by the agent is accurate and directly answers the user's question about the dollar price of Ethereum on May 6th, 2024.", additional_kwargs={}, raw={'id': 'chatcmpl-9Pz5a7j27575drasiQvGWFQgkTrE4', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="has_error: No\nnew_question: N/A\nexplanation: The response provided by the agent is accurate and directly answers the user's question about the dollar price of Ethereum on May 6th, 2024.", role='assistant', function_call=None, tool_calls=None))], 'created': 1715980054, 'model': 'gpt-3.5-turbo-0125', 'object': 'chat.completion', 'system_fingerprint': None, 'usage': CompletionUsage(completion_tokens=45, prompt_tokens=105, total_tokens=150)}, logprobs=None, delta=None))